In [13]:
from conga2 import *
import numpy as np
import collections as co

from igraph import *
import operator
import itertools
import argparse

import random

In [30]:
teams = 10
players_per_team = 10
min_stayed = 0.7 * players_per_team
transfer_windows = 10

In [31]:
def GenerateFootballGraph(teams, players_per_team, min_stayed, transfer_windows):

    #Generate team_labels
    team_labels = []
    for i in range(0,teams):
        lst = [*range(i*players_per_team, (i+1)*players_per_team)]
        team_labels.append(lst)

    #Generate graph
    test_graph = Graph()
    test_graph.add_vertices(teams*players_per_team)
    for t in range(0,teams):
        for i in range(0,players_per_team):
            for j in range(i,players_per_team-1):
                test_graph.add_edges([(i + t * players_per_team ,j+ 1 + t * players_per_team)])


    #Make transfers
    for times in range(0,transfer_windows):
        for i in range(0,teams-1):
            players_chosen = int((len(team_labels[i]) - min_stayed) / 2)
            for j in range(0,players_chosen):
                player_to_transfer = (team_labels[i][random.randint(0, len(team_labels[i]) - 1)])
                team_labels[i].remove(player_to_transfer)
                new_team = random.randint(0,teams-1)
                if new_team == i:
                    new_team += 1
                    new_team %= teams
                team_labels[new_team].append(player_to_transfer)
                for k in team_labels[new_team]:
                    test_graph.add_edges([(player_to_transfer,k)])
    #Graph.simplify(test_graph)
    return test_graph

In [1]:
g = GenerateFootballGraph(teams, players_per_team, min_stayed, transfer_windows)
len(g.components())
print(g)

In [33]:
result = conga(g,optimal_count=5)

In [2]:
result.pretty_print_cover(result.optimal_count, label="CONGA_index")


In [4]:
from time import sleep
from typing import Counter
from igraph import *
import igraph as ig
import csv
import collections
import random

def delete_comments(csvfile):
    for row in csvfile:
        raw = row.split('#')[0].strip()
        if raw:
            yield raw

def read_graph( edges_file):
    print('reading from file')
    edges = []
   

    with open(edges_file , 'r') as csvfile:
        csvreader = csv.reader(delete_comments(csvfile), delimiter=',')
        edges = list(csvreader)

    print('edges setup')
    edgevertices = list([edge[0] for edge in edges] + [edge[1] for edge in edges])

    print('edgevertices')
    edgevertices = list(dict.fromkeys(edgevertices))
  
    print('inverse_indicies')
    inverse_indicies = dict(zip(edgevertices, range(len(edgevertices))))

    g = Graph()

    g.add_vertices(len(edgevertices))
    print('no vertices   ' + str(len(edgevertices)))
    g.vs["id"] = [vertex for vertex in edgevertices]
    g.vs["label"] = g.vs["id"]
    g.vs["vertex_size"] = 8
    g.vs["newlpalabel"] = 0
    g.es["weight"] = 2
    g.vs["vertex_color"] = 0

    for vert in g.vs:
        vert["vertex_color"] = f'rgb({random.randint(0, 255)}, {random.randint(0, 255)}, {random.randint(0, 255)})'
    g.vs["lpalabel"] = list(zip(range(len(edgevertices)), g.vs["vertex_color"]))

    print("adding edges")
    try:
        for edge in edges:
            g.add_edge(inverse_indicies[edge[0]], inverse_indicies[edge[1]])
    except:
        print("Error occured during graph creating")
        quit()

    g.es["weigth"] = 1
    print("graph created")
    return g

In [5]:
swiss = read_graph("..\scrapers\Swiss-Super-League-StatsEdges.csv")

reading from file
edges setup
edgevertices
inverse_indicies
no vertices   585
adding edges
graph created


In [14]:
Graph.simplify(swiss)
SwissLeague = conga(swiss,optimal_count=10)

In [3]:
#SwissLeague.pretty_print_cover(SwissLeague.optimal_count, label="CONGA_index")

cover = SwissLeague._covers[10]

list_communities = []
for i in range(0, 10):
    list_communities.append(cover._clusters[i])

print(list_communities)
len(list_communities)

'\ncover = SwissLeague._covers[10]\n\nlist_communities = []\nfor i in range(0, 10):\n    list_communities.append(cover._clusters[i])\n\nprint(list_communities)\nlen(list_communities)\n'

In [4]:
croatia = read_graph("..\scrapers\cr-StatsEdges.csv")
Graph.simplify(croatia)
print(croatia)

In [43]:
Graph.simplify(croatia)
CroatiaLeague = conga(croatia,optimal_count=10)

In [5]:
cover = CroatiaLeague._covers[10]

list_communities = []
for i in range(0, 10):
    list_communities.append(cover._clusters[i])

#print(list_communities)
#len(list_communities)
SwissLeague.pretty_print_cover(SwissLeague.optimal_count, label="CONGA_index")